In [1]:
import torch
# model_silero_te, example_texts, languages, punct, apply_te = torch.hub.load(repo_or_dir='snakers4/silero-models',
#                                                                   model='silero_te')
import re
import string
from nltk.tokenize import word_tokenize,TreebankWordTokenizer


/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import soundfile
# load model and processor
import os
# import librosa
import jiwer
import string
import csv
from autocorrect import Speller
spell = Speller()
from transformers import pipeline

# fix_spelling = pipeline("text2text-generation",model="oliverguhr/spelling-correction-english-base")


def get_gold_hyp_lists(gold_path,test_path,septype):
    def create_hyp_sentence_dictionary(filename):
        sentence_dict = {}
        with open(filename, 'r') as file:
            lines = file.readlines()
            for idx,line in enumerate(lines):
                line = line.strip()
                if line:
                    sentence_id, sentence = line.split(septype,1)
                    # sentence_id=sentence_id+".wav" # for AMI
                    sentence_dict[sentence_id] = sentence
        return sentence_dict

    def create_gold_sentence_dictionary(filename):
        sentence_dict = {}
        with open(filename, 'r') as file:
            for line in file:
                parts = line.split(' ', 1) # Libri
                # parts = line.split(',', 1) # AMI
                sentence_id, sentence = parts
                if sentence.strip()!='':
                    sentence_dict[sentence_id] = sentence.strip()
        return sentence_dict
    
    def read_csv_to_dict(file_path):
        result_dict = {}
        with open(file_path, 'r') as csvfile:
            csv_reader = csv.reader(csvfile)
            for row in csv_reader:
                key, value = row[0], row[1]
                key=key.split('/')[1]
                # print(key,value)
                # break
                result_dict[key] = value
        return result_dict
    
    gold_dict = create_gold_sentence_dictionary(gold_path)
    # gold_dict = read_csv_to_dict(gold_path)
    test_dict = create_hyp_sentence_dictionary(test_path)
    
    gold_list = []
    test_list = []
    for test_key, test_value in test_dict.items():
        if test_key in gold_dict:
            gold = gold_dict[test_key]
            gold_list.append(gold)
            test_list.append(test_value)
    print(len(test_list))
    return gold_list,test_list

def add_space_before_punctuation(text):
    # List of punctuation marks
    punctuation_marks = string.punctuation

    # Iterate over each character in the text
    modified_text = ''
    for char in text:
        if char in punctuation_marks:
            modified_text += ' ' + char
        else:
            modified_text += char

    return modified_text

def add_space_before_punctuation(text):
    # List of punctuation marks
    punctuation_marks = string.punctuation

    # Iterate over each character in the text
    modified_text = ''
    for char in text:
        if char in punctuation_marks:
            modified_text += ' ' + char
        else:
            modified_text += char

    return modified_text

def add_space(reference_list, hypothesis_list):
    reference = []
    hypothesis = []
    for ref, hyp in zip(reference_list, hypothesis_list):
        # Preprocess sentences to keep punctuation as separate words
        reference.append(add_space_before_punctuation(ref))
        hypothesis.append(add_space_before_punctuation(hyp))
        # print(reference,hypothesis)
        # Calculate WER
       
    #     wer = jiwer.wer(reference, hypothesis)
    #     # print(wer)
    #     wer_scores.append(wer)
    # wer_mean=sum(wer_scores) / len(wer_scores)
    
    return reference,hypothesis

def get_detail_wer(gold_list,test_list):
    out = jiwer.process_words(
            gold_list,
                test_list,
            )
    print("wer: ",out.wer*100)
    print("hits: ",out.hits)
    print("substitutions: ",out.substitutions)
    print("insertions: ",out.insertions)
    print("deletions: ",out.deletions)
    print("total words: ",out.hits+out.substitutions+out.deletions)

# def get_t_p_wer(substitutions, insertions,deletions,total_words_punc, P_WER):
#     t_p_wer=(substitutions+insertions+deletions)/total_words_punc
#     print("Topline of P-WER is ",t_p_wer*100)
#     print("Relative PER is ", P_WER-t_p_wer*100)

def get_c_p_er_fr_wer(wer,C_WER,P_WER,clean=True):
    if clean:
        total_words=50082
        total_words_punt=57312
        cased_words=3849
        p_num = 9709
    else:
        total_words=48488
        total_words_punt=55896
        cased_words=4231
        p_num = 9408
    error_words = wer*total_words
    error_c = C_WER*total_words
    cer=(error_c-error_words)/cased_words
    error_p = P_WER*total_words_punt
    per=(error_p-error_words)/p_num
    # print("Topline of P-WER is ",t_p_wer)
    print("Relative CER is ", cer)
    print("Relative PER is ", per)

def get_c_p_er_fr_wer_ami(wer,C_WER,P_WER):
    # if clean:
    total_words=94006
    total_words_punt=110266
    cased_words=7626
    p_num = 16260
    error_words = wer*total_words
    error_c = C_WER*total_words
    cer=(error_c-error_words)/cased_words
    error_p = P_WER*total_words_punt
    per=(error_p-error_words)/p_num
    print("Relative CER is ", cer)
    print("Relative PER is ", per)

def get_cased_word(text_list):
    capitalized_word_count = 0

    # Iterate through the list of text
    for text in text_list:
        # Split the text into words
        words = text.split()
        
        # Check each word in the text
        for word in words:
            # Check if the word is capitalized (first character is uppercase)
            if word.isalpha() and word[0].isupper():
                capitalized_word_count += 1
    print("Cased words number is ", capitalized_word_count)

In [3]:
gold_list,test_list = get_gold_hyp_lists("/Users/ccui/Desktop/dictation/books-transcription/test-clean-rich-book.txt","clean.txt",',')
# get_cased_word(gold_list)
gold_list,test_list= add_space(gold_list,test_list)
print("CP-WER")
total_wer = get_detail_wer(gold_list, test_list)
gold_rc= jiwer.ToLowerCase()(gold_list)
test_rc = jiwer.ToLowerCase()(test_list)
print("P-WER")
wer_rc = get_detail_wer(gold_rc, test_rc)
# print("WER removing case:", wer_rc*100)
gold_rp= jiwer.RemovePunctuation()(gold_list)
test_rp = jiwer.RemovePunctuation()(test_list)
# gold_rp,test_rp= add_space(gold_rp,test_rp)
print("C-WER")
wer_rp = get_detail_wer(gold_rp, test_rp)
# print("WER removing punctuation:", wer_rp*100)
gold_rc_rp= jiwer.RemovePunctuation()(gold_rc)
test_rc_rp = jiwer.RemovePunctuation()(test_rc)
wer_rc_rp = get_detail_wer(gold_rc_rp, test_rc_rp)

2488
CP-WER
wer:  11.667713567839195
hits:  51008
substitutions:  4934
insertions:  383
deletions:  1370
total words:  57312
P-WER
wer:  7.309115019542155
hits:  53510
substitutions:  2428
insertions:  387
deletions:  1374
total words:  57312
C-WER
wer:  7.607523661195639
hits:  46414
substitutions:  3513
insertions:  142
deletions:  155
total words:  50082
wer:  2.537837945768939
hits:  48953
substitutions:  974
insertions:  142
deletions:  155
total words:  50082


In [4]:
get_c_p_er_fr_wer(wer=2.3421588594704685,C_WER=6.6750529132223155,P_WER=7.122417643774427,clean=True)

Relative CER is  56.378280072746165
Relative PER is  29.961891028942212


In [4]:
gold_list,test_list = get_gold_hyp_lists("/Users/ccui/Desktop/dictation/books-transcription/test-other-rich-book.txt","other.txt",',')
# get_cased_word(gold_list)
gold_list,test_list= add_space(gold_list,test_list)
print("CP-WER")
total_wer = get_detail_wer(gold_list, test_list)
# print("Total WER:", total_wer*100)
gold_rc= jiwer.ToLowerCase()(gold_list)
test_rc = jiwer.ToLowerCase()(test_list)
print("P-WER")
wer_rc = get_detail_wer(gold_rc, test_rc)
# print("WER removing case:", wer_rc*100)
gold_rp= jiwer.RemovePunctuation()(gold_list)
test_rp = jiwer.RemovePunctuation()(test_list)
# gold_rp,test_rp= add_space(gold_rp,test_rp)
print("C-WER")
wer_rp = get_detail_wer(gold_rp, test_rp)
# print("WER removing punctuation:", wer_rp*100)
gold_rc_rp= jiwer.RemovePunctuation()(gold_rc)
test_rc_rp = jiwer.RemovePunctuation()(test_rc)
print("WER")
wer_rc_rp = get_detail_wer(gold_rc_rp, test_rc_rp)

2727
CP-WER
wer:  14.432159725203949
hits:  48326
substitutions:  6093
insertions:  497
deletions:  1477
total words:  55896
P-WER
wer:  9.914841849148418
hits:  50851
substitutions:  3568
insertions:  497
deletions:  1477
total words:  55896
C-WER
wer:  10.955287906286092
hits:  43417
substitutions:  4744
insertions:  241
deletions:  327
total words:  48488
WER
wer:  5.6550074245174065
hits:  45987
substitutions:  2174
insertions:  241
deletions:  327
total words:  48488


In [10]:
get_c_p_er_fr_wer(wer=5.49208051476654,C_WER=8.177280976736512,P_WER=9.750250465149565,clean=False)

Relative CER is  30.772866934530843
Relative PER is  29.62372448979592


In [4]:
gold_list,test_list = get_gold_hyp_lists("/Users/ccui/Desktop/dictation/vasr/calculate_wer/ami_test_gold_process.txt","ihm.txt",',')
# get_cased_word(gold_list)
gold_list,test_list= add_space(gold_list,test_list)
print("CP-WER")
total_wer = get_detail_wer(gold_list, test_list)
# print("Total WER:", total_wer*100)
gold_rc= jiwer.ToLowerCase()(gold_list)
test_rc = jiwer.ToLowerCase()(test_list)
print("P-WER")
wer_rc = get_detail_wer(gold_rc, test_rc)
# print("WER removing case:", wer_rc*100)
gold_rp= jiwer.RemovePunctuation()(gold_list)
test_rp = jiwer.RemovePunctuation()(test_list)
print("C-WER")
wer_rp = get_detail_wer(gold_rp, test_rp)
# print("WER removing punctuation:", wer_rp*100)
gold_rc_rp= jiwer.RemovePunctuation()(gold_rc)
test_rc_rp = jiwer.RemovePunctuation()(test_rc)
wer_rc_rp = get_detail_wer(gold_rc_rp, test_rc_rp)

9474
CP-WER
wer:  51.19619828414924
hits:  57856
substitutions:  29649
insertions:  4042
deletions:  22761
total words:  110266
P-WER
wer:  48.772060290570074
hits:  60599
substitutions:  26836
insertions:  4112
deletions:  22831
total words:  110266
C-WER
wer:  43.63444886496606
hits:  55024
substitutions:  24781
insertions:  2037
deletions:  14201
total words:  94006
wer:  40.66336191306938
hits:  57850
substitutions:  21922
insertions:  2070
deletions:  14234
total words:  94006


In [5]:
get_c_p_er_fr_wer_ami(wer=40.66336191306938,C_WER=43.63444886496606,P_WER=48.772060290570074)

Relative CER is  36.62470495672692
Relative PER is  95.6519065190652
